<a href="https://colab.research.google.com/github/ncoop57/t5_overlap/blob/main/overlap_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [15]:
%%bash
# pip install sentencepiece gdown statsmodels # git+https://github.com/ncoop57/cute_deltas
cp -r /gdrive/MyDrive/T5-Extension/datasets .
cp -r /gdrive/MyDrive/T5-Extension/DataSnooping_Analysis_Data .
# gdown https://drive.google.com/uc?id=1KBVkhFZ80i1PW80aZexFGvJjyEClelX1

In [16]:
! zip -r datasets.zip datasets
! zip -r DataSnooping_Analysis_Data.zip DataSnooping_Analysis_Data

  adding: datasets/ (stored 0%)
  adding: datasets/tsv/ (stored 0%)
  adding: datasets/tsv/pre-training/ (stored 0%)
  adding: datasets/tsv/pre-training/pre-training.tsv (deflated 82%)
  adding: datasets/tsv/fine-tuning/ (stored 0%)
  adding: datasets/tsv/fine-tuning/CS/ (stored 0%)
  adding: datasets/tsv/fine-tuning/CS/test.tsv (deflated 81%)
  adding: datasets/tsv/fine-tuning/CS/eval.tsv (deflated 81%)
  adding: datasets/tsv/fine-tuning/CS/training.tsv (deflated 81%)
  adding: datasets/tsv/fine-tuning/AGabs/ (stored 0%)
  adding: datasets/tsv/fine-tuning/AGabs/test.tsv (deflated 87%)
  adding: datasets/tsv/fine-tuning/AGabs/eval.tsv (deflated 87%)
  adding: datasets/tsv/fine-tuning/AGabs/training.tsv (deflated 90%)
  adding: datasets/tsv/fine-tuning/AGraw/ (stored 0%)
  adding: datasets/tsv/fine-tuning/AGraw/test.tsv (deflated 78%)
  adding: datasets/tsv/fine-tuning/AGraw/eval.tsv (deflated 78%)
  adding: datasets/tsv/fine-tuning/AGraw/training.tsv (deflated 78%)
  adding: datasets/t

In [17]:
! mv datasets.zip DataSnooping_Analysis_Data.zip /gdrive/MyDrive/T5-Extension/

In [ ]:
! pip install sentencepiece gdown statsmodels

     |████████████████████████████████| 1.2MB 5.6MB/s 


In [ ]:
import pandas as pd

from pathlib import Path

fine_tune_path = Path("datasets/tsv/fine-tuning")

def get_agab_dfs():
    trn_agab_df = pd.read_csv(
        fine_tune_path/"AGabs/training.tsv", sep="\t", 
        names=["input", "target"]
    )
    tst_agab_df = pd.read_csv(
        "DataSnooping_Analysis_Data/AGabs.csv", index_col=0
    ).sort_values("IS_Perfect")

    return trn_agab_df, tst_agab_df

def get_agraw_dfs():
    trn_agraw_df = pd.read_csv(
        fine_tune_path/"AGraw/training.tsv", sep="\t",
        names=["input", "target"]
    )
    tst_agraw_df = pd.read_csv(
        "DataSnooping_Analysis_Data/AGraw.csv", index_col=0
    ).sort_values("IS_Perfect")

    return trn_agraw_df, tst_agraw_df

def get_bfsm_dfs():
    trn_bfsm_df = pd.read_csv(
        "datasets/tsv/fine-tuning/BFsmall/training.tsv", sep="\t",
        names=["input", "target"]
    )
    tst_bfsm_df = pd.read_csv(
        "DataSnooping_Analysis_Data/BFsmall.csv", index_col=0
    ).sort_values("IS_Perfect")

    return trn_bfsm_df, tst_bfsm_df

def get_bfmed_dfs():
    trn_bfmed_df = pd.read_csv(
        "datasets/tsv/fine-tuning/BFmedium/training.tsv", sep="\t",
        names=["input", "target"]
    )
    tst_bfmed_df = pd.read_csv(
        "DataSnooping_Analysis_Data/BFmedium.csv", index_col=0
    ).sort_values("IS_Perfect")

    return trn_bfmed_df, tst_bfmed_df

def get_codesum_dfs():
    trn_codesum_df = pd.read_csv(
        "datasets/tsv/fine-tuning/CS/training.tsv", sep="\t",
        names=["input", "target"]
    )
    tst_codesum_df = pd.read_csv(
        "DataSnooping_Analysis_Data/CodeSummarization.csv", index_col=0
    ).sort_values("BLEU")

    return trn_codesum_df, tst_codesum_df

def get_muts_dfs():
    trn_muts_df = pd.read_csv(
        "datasets/tsv/fine-tuning/MG/training.tsv", sep="\t",
        names=["input", "target"]
    )
    tst_muts_df = pd.read_csv(
        "DataSnooping_Analysis_Data/Mutants.csv", index_col=0
    ).sort_values("BLEU")

    return trn_muts_df, tst_muts_df

def sample_bst_wrst(df, pop=1_000, n=100):
    bst = df.tail(pop)
    wrst = df.head(pop)

    return bst.sample(n), wrst.sample(n)

In [ ]:
# This code was taken from https://gist.github.com/kylebgorman/1081951/bce3de986e4b05fc0b63d4d9e0cfa4bde6664365
def _dist(A, B, insertion, deletion, substitution):
    D = np.zeros((len(A) + 1, len(B) + 1))
    for i in range(len(A)):
        D[i + 1][0] = D[i][0] + deletion
    for j in range(len(B)):
        D[0][j + 1] = D[0][j] + insertion
    for i in range(len(A)): # fill out middle of matrix
        for j in range(len(B)):
            if A[i] == B[j]:
                D[i + 1][j + 1] = D[i][j] # aka, it's free.
            else:
                D[i + 1][j + 1] = min(D[i + 1][j] + insertion,
                                      D[i][j + 1] + deletion,
                                      D[i][j]     + substitution)
    return D

def levenshtein_distance(l1, l2, normalize=False):
    dist = _dist(l1, l2, 1, 1, 1)[-1][-1]
    if normalize:
        return 1. - dist / max(len(l1), len(l2))
    else:
        return dist

In [ ]:
# From https://github.com/neilernst/cliffsDelta
from __future__ import division


def cliffsDelta(lst1, lst2, **dull):

    """Returns delta and true if there are more than 'dull' differences"""
    if not dull:
        dull = {'small': 0.147, 'medium': 0.33, 'large': 0.474} # effect sizes from (Hess and Kromrey, 2004)
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats, x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] < x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
    d = (more - less) / (m*n)
    size = lookup_size(d, dull)
    return d, size


def lookup_size(delta: float, dull: dict) -> str:
    """
    :type delta: float
    :type dull: dict, a dictionary of small, medium, large thresholds.
    """
    delta = abs(delta)
    if delta < dull['small']:
        return 'negligible'
    if dull['small'] <= delta < dull['medium']:
        return 'small'
    if dull['medium'] <= delta < dull['large']:
        return 'medium'
    if delta >= dull['large']:
        return 'large'


def runs(lst):
    """Iterator, chunks repeated values"""
    for j, two in enumerate(lst):
        if j == 0:
            one, i = two, 0
        if one != two:
            yield j - i, one
            i = j
        one = two
    yield j - i + 1, two

In [ ]:
import numpy as np

from joblib import Parallel, delayed
from scipy.stats import mannwhitneyu
import scipy.stats as st
from statistics import mean, median, stdev

def get_dists(trn, tst):
    import sentencepiece as spm
    s = spm.SentencePieceProcessor(model_file='dl4se_vocab.model')
    s.encode("public static void main", out_type=str)
    dists = Parallel(n_jobs=-1)(
        delayed(levenshtein_distance)(s.encode(i), s.encode(j))
        for i in trn for j in tst
    )
    
    return dists

# From https://stackoverflow.com/a/51288518/5768407 by Yetti
def ci_overlap(start1, end1, start2, end2):
    """how much does the range (start1, end1) overlap with (start2, end2)"""
    return max(max((end2-start1), 0) - max((end2-end1), 0) - max((start2-start1), 0), 0)

def run_experiments(data_func, n_exp=100, pop=1_000, n_samp=100, alpha = 0.95):
    bst_dists = []
    wrst_dists = []

    trn_df, tst_df = data_func()
    for _ in range(n_exp):
        (trn_bst, trn_wrst), (tst_bst, tst_wrst) = sample_bst_wrst(trn_df, pop, n_samp), sample_bst_wrst(tst_df, pop, n_samp)
        bst_dists.extend(get_dists(trn_bst["target"].values, tst_bst["Groundtruth"].values))
        wrst_dists.extend(get_dists(trn_wrst["target"].values, tst_wrst["Groundtruth"].values))
    # change to confidence intervals
    # mannwhitneyu_results = mannwhitneyu(results["best"]["dists"], results["worst"]["dists"], alternative="less")
    # cliffs_results = cliffsDelta(results["best"]["dists"], results["worst"]["dists"])
    bst_ci = st.t.interval(
        alpha=alpha, df=len(bst_dists)-1,
        loc=np.mean(bst_dists),
        scale=st.sem(bst_dists)
    )
    wrst_ci = st.t.interval(
        alpha=alpha, df=len(wrst_dists)-1,
        loc=np.mean(wrst_dists),
        scale=st.sem(wrst_dists)
    )

    results = {
        "best": {
            "dists": bst_dists,
            "mean": mean(bst_dists),
            "median": median(bst_dists),
            "stdev": stdev(bst_dists),
            "ci": bst_ci,
        },
        "worst": {
            "dists": wrst_dists,
            "mean": mean(wrst_dists),
            "median": median(wrst_dists),
            "stdev": stdev(wrst_dists),
            "ci": wrst_ci,
        },
        "overlap": ci_overlap(*bst_ci, *wrst_ci),
        "ci_alpha": alpha,
    }
    # cm = sms.CompareMeans(sms.DescrStatsW(results["best"]["dists"]), sms.DescrStatsW(results["worst"]["dists"]))
    # ci = cm.tconfint_diff(usevar='unequal')

    return results

In [ ]:
import json

def data_snooping_analysis(output_path, n_exp=30, pop=1_000, n_samp=100):
    datasets = [
        # ("agab", get_agab_dfs), ("agraw", get_agraw_dfs),
        # ("bfsm", get_bfsm_dfs), 
        ("bfmed", get_bfmed_dfs), # need to run this with fewer samples/experiments due to size
        # ("codesum", get_codesum_dfs), ("muts", get_muts_dfs)
    ]

    for name, ds in datasets:
        results = run_experiments(ds, n_exp=n_exp, pop=pop, n_samp=n_samp)
        with open(output_path/f"{name}_results.json", 'w') as json_file:
            json.dump(results, json_file)

In [ ]:
%%time
# for bfmed
output_path = Path("/gdrive/MyDrive/T5-Extension/DataSnooping_Analysis_Data")
data_snooping_analysis(output_path, n_exp=1, pop=1_000, n_samp=100)

CPU times: user 47.9 s, sys: 1.38 s, total: 49.2 s
Wall time: 11min 55s


In [ ]:
%%time
output_path = Path("/gdrive/MyDrive/T5-Extension/DataSnooping_Analysis_Data")
data_snooping_analysis(output_path, n_exp=30, pop=1_000, n_samp=100)

In [ ]:
ls

sample_data/


In [ ]:
%%time
results = run_experiments(get_agraw_dfs, n_exp=1, pop=1_000, n_samp=100)

CPU times: user 11.5 s, sys: 247 ms, total: 11.7 s
Wall time: 42.2 s


In [ ]:
results["best"]["ci"]

(23.55650225893392, 24.35429774106608)

In [ ]:
results["best"]["ci"]

(23.185239365108536, 23.909160634891464)

In [ ]:
bst_ci, wrst_ci, overlap

((19.302129843465664, 19.722070156534336),
 (24.004370813763796, 24.604229186236207),
 0)

In [ ]:
mannwhitneyu_results, cliffs_results

(MannwhitneyuResult(statistic=44739502.0, pvalue=2.6755838935035946e-38),
 (-0.10520996, 'negligible'))

In [ ]:
%%time
results, mannwhitneyu_results, cliffs_results = run_experiments(get_agab_dfs, n = 1)

CPU times: user 11.5 s, sys: 636 ms, total: 12.1 s
Wall time: 4min 12s


In [ ]:
mannwhitneyu_results, cliffs_results

(MannwhitneyuResult(statistic=38004042.5, pvalue=3.6365152894718975e-190),
 (-0.23991915, 'small'))

In [ ]:
%%time
results = run_experiments(trn_agraw_df, tst_agraw_df, n = 1)

CPU times: user 12.4 s, sys: 481 ms, total: 12.9 s
Wall time: 5min 38s


In [ ]:
from scipy.stats import mannwhitneyu

mannwhitneyu(results["best"]["dists"], results["worst"]["dists"], alternative="less")

MannwhitneyuResult(statistic=48895852.0, pvalue=0.003416991123538754)

In [ ]:
cliffsDelta(results["best"]["dists"], results["worst"]["dists"])

(-0.02208296, 'negligible')

In [ ]:
results

In [ ]:
%%time
trn, tst = trn_bst["target"].values, tst_bst["Groundtruth"].values

avg, med, std = get_overlap(trn, tst)

CPU times: user 3.15 s, sys: 140 ms, total: 3.29 s
Wall time: 1min 30s


In [ ]:
avg, med, std

(47.4261, 32.0, 43.71244692981232)

In [ ]:
%%time
trn, tst = trn_wrst["target"].values, tst_wrst["Groundtruth"].values

avg, med, std = get_overlap(trn, tst)

CPU times: user 5.12 s, sys: 248 ms, total: 5.37 s
Wall time: 1min 51s


In [ ]:
avg, med, std

(51.6158, 44.0, 30.266013835973705)

In [ ]:
df = pd.DataFrame(zip(trn_agraw_df.head(10)["input"].values, tst_agraw_df.head()["Input"].values), columns=["train_input", "test_input"])

In [ ]:
df.apply(lambda x: levenshtein_distance(x["train_input"], x["test_input"]), axis=1)

0    1269.0
1     559.0
2     351.0
3     648.0
4     260.0
dtype: float64

In [ ]:
%%time
from joblib import Parallel, delayed

trn, tst = trn_agraw_df.head()["input"].values, tst_agraw_df.head()["Input"].values

parallel_dists = Parallel(n_jobs=2)(delayed(levenshtein_distance)(i, j) for i in trn for j in tst)

CPU times: user 179 ms, sys: 16 ms, total: 195 ms
Wall time: 20.6 s


In [ ]:
%%time
from joblib import Parallel, delayed

trn, tst = trn_agraw_df.head()["input"].values, tst_agraw_df.head()["Input"].values

parallel_dists = Parallel(n_jobs=-1)(delayed(levenshtein_distance)(i, j) for i in trn for j in tst)

CPU times: user 192 ms, sys: 15.9 ms, total: 208 ms
Wall time: 21.1 s


In [ ]:
%%time

dists = []
for i in trn:
    for j in tst:
        dists.append(levenshtein_distance(i, j))

CPU times: user 27.9 s, sys: 61.5 ms, total: 28 s
Wall time: 28.1 s


In [ ]:
parallel_dists == dists

True

In [ ]:
len(dists)

25

In [ ]:
! lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           AuthenticAMD
CPU family:          23
Model:               49
Model name:          AMD EPYC 7B12
Stepping:            0
CPU MHz:             2250.000
BogoMIPS:            4500.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a 

In [ ]:
trn_agraw_df, tst_agraw_df = get_agraw_dfs(); tst_agraw_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,testgetlibrariesdoesdeduplication ( ) { when (...,org . junit . assert . assertarrayequals ( new...,"org. junit. assert. assertequals ( 0, categori...",False
10501,testclosewithoutwritinganything ( ) { com . go...,org . junit . assert . asserttrue ( getfilecon...,"org. junit. assert. assertequals ( 0, channel....",False
10502,testjavaversionloginfoandoutput ( ) { org . ze...,org . junit . assert . assertfalse ( org . apa...,org. junit. assert. assertnotnull ( str ),False
10503,checktime10 ( ) { com . alibaba . fastjson . p...,"org . junit . assert . assertequals ( false , ...","org. junit. assert. assertequals ( true, retval )",False
10509,testcanparserule ( ) { org . cogroo . tools . ...,"org . junit . assert . assertequals ( 1 , acce...","org. junit. assert. assertequals ( 1, access. ...",False


In [ ]:
trn_bfmed_df, tst_bfmed_df = get_bfmed_dfs(); tst_bfmed_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,public void method_1 ( ) { for ( int i = ( ( v...,public void method_1 ( ) { for ( int i = ( ( v...,public void method_1 ( ) { for ( int i = ( ( v...,False
4319,public void method_1 ( android.view.view var_1...,public void method_1 ( android.view.view var_1...,public void method_1 ( android.view.view var_1...,False
4318,public type_1 method_1 ( int var_1 ) { if ( ! ...,public type_1 method_1 ( int var_1 ) { if ( ! ...,public type_1 method_1 ( int var_1 ) { if (! (...,False
4317,public void method_1 ( ) { if ( ( ( var_1 ) ==...,public void method_1 ( ) { if ( ( ( var_1 ) ==...,public void method_1 ( ) { if ( ( ( var_1 ) ==...,False
4316,public void method_1 ( ) { java.util.iterator ...,public void method_1 ( ) { java.util.iterator ...,public void method_1 ( ) { java.util.iterator ...,False


In [ ]:
trn_codesum_df, tst_codesum_df = get_codesum_dfs(); tst_codesum_df.head()

,Input,Groundtruth,Output,IS_Perfect,BLEU,ROUGE-R,ROUGE-P,ROUGE-F
78023,public object adapter get object adapter retur...,lazily looks up,returns the adapter that this object is associ...,False,0.0,0.0,0.0,0.000001
78025,public object specification get type of specif...,overridable for submodels eg,returns the type of specification from the giv...,False,0.0,0.0,0.0,0.000001
78022,public object specification get specification ...,lazy loaded from,return the specification,False,0.0,0.0,0.0,0.000001
45311,public boolean inform return true,request if this event is an internal or extern...,inform the user of the change,False,0.0,0.0,0.0,0.000001
45316,public v put else v value return condition map...,n this map,store the value for the if condition,False,0.0,0.0,0.0,0.000001


In [ ]:
trn_muts_df, tst_muts_df = get_muts_dfs()

In [ ]:
trn_muts_df.head()

,target
input,
private void not ( short VAR_1 ) { short VAR_2 = VAR_3 [ VAR_1 ] ; this . VAR_3 [ VAR_1 ] = ( ( short ) ( ~ VAR_2 ) ) ; },private void not ( short VAR_1 ) { this . VAR_...
"protected java.util.List < java.lang.String > METHOD_1 ( ) { return java.util.Arrays.asList ( VAR_1 , VAR_2 ) ; }",protected java.util.List < java.lang.String > ...
public java.lang.Integer METHOD_1 ( TYPE_1 VAR_1 ) { VAR_2 . METHOD_2 ( STRING_1 ) ; VAR_2 . METHOD_2 ( ( STRING_2 + ( id ) ) ) ; return super . METHOD_1 ( VAR_1 ) ; },public java.lang.Integer METHOD_1 ( TYPE_1 VAR...
public static TYPE_1 METHOD_1 ( ) { TYPE_1 VAR_1 = new TYPE_1 ( STRING_1 ) ; VAR_1 . METHOD_2 ( TYPE_2 . METHOD_3 ( STRING_2 ) ) ; return VAR_1 ; },public static TYPE_1 METHOD_1 ( ) { TYPE_1 VAR...
public boolean METHOD_1 ( ) { if ( ( ( ( VAR_1 ) >= 0 ) && ( ( VAR_2 ) >= 0 ) ) && ( VAR_3 ) ) return true ; return false ; },public boolean METHOD_1 ( ) { if ( ( ( ( VAR_1...


In [ ]:
tst_muts_df.head()

,Input,Groundtruth,Output,IS_Perfect,BLEU
2947,public boolean update ( ) { boolean var_1 = fa...,public boolean update ( ) { boolean var_1 = fa...,public boolean update ( ) { return false ; },False,0.029103
7456,public java.util.date method_1 ( ) { java.util...,public java.util.date method_1 ( ) { java.util...,public java.util.date method_1 ( ) { return da...,False,0.065435
2026,public void method_1 ( ) { type_1 var_1 = ( ( ...,public void method_1 ( ) { type_1 var_1 = ( ( ...,public void method_1 ( ) { method_3 ( ) ; },False,0.068947
6654,public type_1 [ ] method_1 ( java.lang.string ...,public void method_1 ( ) { },public type_1 [ ] method_1 ( java.lang.string ...,False,0.096321
2250,public void method_1 ( ) throws java.lang.exce...,public void method_1 ( ) { },public void method_1 ( ) throws java.lang.exce...,False,0.109680


In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='dl4se_vocab.model')

In [ ]:
s.encode("public static void main", out_type=str)

['▁public', '▁static', '▁void', '▁main']

In [ ]:
import pandas as pd

agab_df = pd.read_csv("DataSnooping_Analysis_Data/AGabs.csv", index_col=0); agab_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,method_0 ( ) { org . apache . commons . ident_...,org . junit . assert . assertequals ( expected...,"org. junit. assert. assertequals ( expected, e...",True
1,method_0 ( ) { org . kaazing . ident_0 . filte...,org . junit . assert . assertequals ( expected...,"org. junit. assert. assertequals ( expected, b...",True
2,method_0 ( ) { org . kaazing . ident_0 . trans...,"org . junit . assert . assertequals ( 3 , iden...","org. junit. assert. assertequals ( 1, ident_8....",False
3,method_0 ( ) { org . apache . commons . ident_...,org . junit . assert . assertequals ( expected...,"org. junit. assert. assertequals ( expected, e...",True
4,method_0 ( ) { final org . apache . ident_0 . ...,org . junit . assert . assertfalse ( ident_6 ....,"org. junit. assert. assertequals ( 1, ident_6....",False


In [ ]:
agraw_df = pd.read_csv("DataSnooping_Analysis_Data/AGraw.csv", index_col=0); agraw_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,testgetlibrariesdoesdeduplication ( ) { when (...,org . junit . assert . assertarrayequals ( new...,"org. junit. assert. assertequals ( 0, categori...",False
1,getuserswaitingnotificationnowatchexpectemptyl...,"org . junit . assert . assertequals ( 0 , user...","org. junit. assert. assertequals ( 0, users. s...",True
2,testcreatornull ( ) { de . sstoehr . harreader...,org . junit . assert . assertnotnull ( log . g...,org. junit. assert. assertnotnull ( log. getcr...,True
3,shouldcorrectlycalculatemutationscoveragewhena...,"org . junit . assert . assertequals ( 100 , th...","org. junit. assert. assertequals ( 1, this. te...",False
4,testbadissuer ( ) { java . security . privatek...,org . junit . assert . asserttrue ( ( ex insta...,org. junit. assert. assertnotnull ( ex ),False


In [ ]:
bfsm_df = pd.read_csv("DataSnooping_Analysis_Data/BFsmall.csv", index_col=0); bfsm_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,private type_1 gettype ( type_2 var_1 ) { type...,private type_1 gettype ( type_2 var_1 ) { type...,private type_1 gettype ( type_2 var_1 ) { type...,False
1,public type_1 method_1 ( ) { type_1 output = v...,public type_1 method_1 ( ) { type_1 output = v...,public type_1 method_1 ( ) { type_1 output = v...,False
2,private void method_1 ( java.lang.class var_1 ...,public void method_1 ( java.lang.class var_1 )...,private void method_1 ( java.lang.class var_1 ...,False
3,public void method_1 ( ) { for ( type_1 var_1 ...,public void method_1 ( ) { method_3 ( ) ; if (...,public void method_1 ( ) { for ( type_1 var_1 ...,False
4,public void method_1 ( ) { type_1 . method_2 (...,private void method_1 ( ) { type_1 . method_2 ...,public void method_1 ( ) { },False


In [ ]:
bfmed_df = pd.read_csv("DataSnooping_Analysis_Data/BFmedium.csv", index_col=0); bfmed_df.head()

,Input,Groundtruth,Output,IS_Perfect
0,public void method_1 ( ) { for ( int i = ( ( v...,public void method_1 ( ) { for ( int i = ( ( v...,public void method_1 ( ) { for ( int i = ( ( v...,False
1,public void method_1 ( type_1 var_1 ) { super ...,public void method_1 ( type_1 var_1 ) { super ...,public void method_1 ( type_1 var_1 ) { super....,False
2,private java.lang.string method_1 ( ) { int va...,private java.lang.string method_1 ( ) { int va...,private java.lang.string method_1 ( ) { int va...,False
3,public type_1 method_1 ( type_1 var_1 ) throws...,public type_1 method_1 ( type_1 var_1 ) throws...,public type_1 method_1 ( type_1 var_1 ) throws...,False
4,protected int method_1 ( int var_1 ) { var_2 ....,protected int method_1 ( int var_1 ) { var_3 =...,protected int method_1 ( int var_1 ) { var_2. ...,False


In [ ]:
codesum_df = pd.read_csv("DataSnooping_Analysis_Data/CodeSummarization.csv", index_col=0); codesum_df.head()

,Input,Groundtruth,Output,IS_Perfect,BLEU,ROUGE-R,ROUGE-P,ROUGE-F
0,public void call throws exception point cost p...,called by the thread executor when the thread ...,run the optimizer,False,0.039589,0.100000,0.100000,0.153847
1,public void initialize display display message...,it is initialized only when display and messag...,initializes the display,False,0.028367,0.090909,0.090909,0.142858
2,private void set columns width int sample len ...,sets the columns width based on an extimation ...,set the width of all columns in the table,False,0.163049,0.400000,0.400000,0.421054
3,private void recalculate column and row count ...,recalculates the column and row count consider...,recalculates column and row count,False,0.357338,0.555556,0.555556,0.714287
4,private void set rows input matrix info arr ma...,adds to the table the matrices definitions as ...,fills the table with the rows of the input matrix,False,0.182574,0.444444,0.444444,0.421054


In [ ]:
muts_df = pd.read_csv("DataSnooping_Analysis_Data/Mutants.csv", index_col=0); muts_df.head()

,Input,Groundtruth,Output,IS_Perfect,BLEU
0,private static java.lang.string method_1 ( jav...,private static java.lang.string method_1 ( jav...,private static java.lang.string method_1 ( jav...,False,0.912382
1,"public synchronized void add ( int index , jav...","public synchronized void add ( int index , jav...","public synchronized void add ( int index , jav...",True,1.000000
2,public void method_1 ( ) { type_1 . method_2 (...,public void method_1 ( ) { super . method_1 ( ...,public void method_1 ( ) { super . method_1 ( ...,True,1.000000
3,protected float method_1 ( float var_1 ) { ret...,protected float method_1 ( float var_1 ) { ret...,protected float method_1 ( float var_1 ) { ret...,False,0.967793
4,public static char [ ] method_1 ( java.lang.st...,public static char [ ] method_1 ( java.lang.st...,public static char [ ] method_1 ( java.lang.st...,False,0.938388
